<a href="https://colab.research.google.com/github/amita-kapoor/UO-Artificial-Intelligence-Cloud-and-Edge-Implementations/blob/master/Excercise_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classification Exercises

For these exercises use the GPU in Google Colab. To enable GPU go to top menu bar in **EDIT** menu go to **NoteBook Settings**. Once you click it a window opens, in the hardware accelerator dropdown menu choose GPU. 

![alt](https://drive.google.com/uc?id=1rZf9pvb5rqY4rFwYqUhdmPkSrzaXBhPg)

### Introduction

We have already learned about Neural Networks and discussed Multilayered Perceptrons in depth. In this exercise, we will be testing our understanding of the underlying concepts with special emphasis to [Hyperparameter tuning](https://towardsdatascience.com/understanding-hyperparameters-and-its-optimisation-techniques-f0debba07568). 

After doing these exercises, you would be able to better understand:

* The architecture of a neural network
* The parameters (training) of a neural network and how they change with changing architecture.
* Hyperparameter tuning: batch size, number of hidden units and optimizers.

We encourage you to work with other hyperparameters as well like learning rate, number of layers, activation functions etc.  And in the end there is an optional exercise, where you can see if what you observe for the MNIST dataset is true for other dataset as well.

The Notebook is divided in three parts: Building the Model, Reading the dataset and Hyperparameters. It contains five exercises in total and one additional optional exercise:

* [Exercise 1](#ex_1)
* [Exercise 2](#ex_2)
* [Exercise 3](#ex_3)
* [Exercise 4](#ex_4)
* [Exercise 5](#ex_5)
* [Optional Exercise](#ex_O)


You have to do all the five exercises. Run the code given with each exercise and write down your answer just below each exercise. Wish you all the best.


### Part 1: Building the model
Below we define a function to built a neural network model using TensorFlow Keras. 

# New section

In [50]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

def built_model(input_shape, n_hidden, nb_classes, optimizer='SGD'):
  '''
  The function builds a fully connected neural network with two hidden layers
  Arguments:
  input_shape: The number of inputs to the neural network
  n_hidden: Number of hidden neurons in the hidden layers
  nb_classes: Number of neurons in the output layer
  optimizer: The optimizer used to train the model. 
  By default we use Stochastic Gradient Descent.
  
  Returns:
  The function returns A model with loss and optimizer defined
  '''  
  model = tf.keras.models.Sequential()
  ## First Hidden layer  
  model.add(keras.layers.Dense(n_hidden,
       input_shape=(input_shape,),
       name='dense_layer', activation='relu'))
    
  ## Second Hidden Layer
  model.add(keras.layers.Dense(n_hidden,
        name='dense_layer_2', activation='relu'))
    
  ## Output Layer  
  model.add(keras.layers.Dense(nb_classes,
        name='dense_layer_3', activation='softmax'))
    
  ## Define loss and optimizer 
  model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return model


<a id='ex_1'></a>
**Exercise 1** What should be the values of the arguments `INPUT_SHAPE`: the number of input units, `N_HIDDEN`: the number of hidden units, and `NB_CLASSES`: the number of output units, if we want to build a model using `built_model` function with the specifications given in the figure:

![](https://drive.google.com/uc?id=1pcj2sHJK6CmhMjUo43AMNBxnU4ixQne3)



To build this network we used TensorFlow Keras `plot_model` function available in `utils` model. You can learn more about the function from [TensorFlow docs](https://www.tensorflow.org/api_docs/python/tf/keras/utils/plot_model). 

In [7]:
# Task to do
INPUT_SHAPE = 5
N_HIDDEN = 10
NB_CLASSES = 2  


## Do not change anything below
assert(INPUT_SHAPE == 5), "Input shape incorrect"
assert(N_HIDDEN == 10), "Number of hidden neurons incorrect"
assert(NB_CLASSES == 2), "Number of output units incorrect"

In [10]:
model = built_model(INPUT_SHAPE, N_HIDDEN,NB_CLASSES)

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 10)                60        
_________________________________________________________________
dense_layer_2 (Dense)        (None, 10)                110       
_________________________________________________________________
dense_layer_3 (Dense)        (None, 2)                 22        
Total params: 192
Trainable params: 192
Non-trainable params: 0
_________________________________________________________________
None


<a id='ex_2'></a>
**Exercise 2** Based on the input, hidden and output units what are the total number of trainable parameters in this model?

In [11]:
# Task to do
trainable_parameters = 192

## Do not change anything below
assert trainable_parameters==model.count_params(), "Your answer is incorrect"
print("Number of trainable parameters in the model are", trainable_parameters)

Number of trainable parameters in the model are 192


Good work! Let us now visualize the summary of the model created. 

In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 10)                60        
_________________________________________________________________
dense_layer_2 (Dense)        (None, 10)                110       
_________________________________________________________________
dense_layer_3 (Dense)        (None, 2)                 22        
Total params: 192
Trainable params: 192
Non-trainable params: 0
_________________________________________________________________


### Part 2: Reading the dataset

We will continue with the MNIST dataset. 

###### Just run the cells in this part of the notebook. Do not change anything.

In [13]:
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [14]:
# Processing the data
assert(len(X_train.shape)==3), "The input data is not of the right shape"
RESHAPED = X_train.shape[1]*X_train.shape[2]

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [15]:
# Data Normalization
X_train, X_test = X_train / 255.0, X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


For the MNIST dataset the number of input and number of output units are fixed. However we can choose different values of hidden units. 

In [16]:
INPUT_SHAPE = RESHAPED
NB_CLASSES = len(set(Y_train))

In [17]:
# one-hot encode
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

### Part 3: Hyperparameters

<a id='ex_3'></a>
**Exercise 3:** The aim of this exercise is to understand the affect of changing number of hidden units on the model performance. Change the number of hidden units, and train the model. Compare the model performance in terms of accuracy. What do you understand from this?

**Answer** Please type your answer here (Double click to edit)

In [19]:
# Task to do choose different values for number of hidden units (minimum five different values)
N_HIDDEN = 10 #Choose a value

In [20]:
## Do not change anything below
model = built_model(INPUT_SHAPE,N_HIDDEN, NB_CLASSES)
history = model.fit(X_train, Y_train,
		batch_size=128, epochs=50,
		verbose=1, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy: {:.2f} %'.format(test_acc*100))

Epoch 1/50
375/375 [==============================] - 4s 3ms/step - loss: 2.2050 - accuracy: 0.2115 - val_loss: 1.6894 - val_accuracy: 0.5091
Epoch 2/50
375/375 [==============================] - 1s 3ms/step - loss: 1.4588 - accuracy: 0.5759 - val_loss: 0.8607 - val_accuracy: 0.8032
Epoch 3/50
375/375 [==============================] - 1s 2ms/step - loss: 0.7964 - accuracy: 0.8013 - val_loss: 0.5601 - val_accuracy: 0.8568
Epoch 4/50
375/375 [==============================] - 1s 3ms/step - loss: 0.5658 - accuracy: 0.8490 - val_loss: 0.4604 - val_accuracy: 0.8749
Epoch 5/50
375/375 [==============================] - 1s 2ms/step - loss: 0.4834 - accuracy: 0.8647 - val_loss: 0.4121 - val_accuracy: 0.8872
Epoch 6/50
375/375 [==============================] - 1s 2ms/step - loss: 0.4221 - accuracy: 0.8818 - val_loss: 0.3861 - val_accuracy: 0.8913
Epoch 7/50
375/375 [==============================] - 1s 2ms/step - loss: 0.4042 - accuracy: 0.8859 - val_loss: 0.3672 - val_accuracy: 0.8959
Epoch 

In [22]:
N_HIDDEN = [10,20,30,40,50]

In [32]:
## Do not change anything below
for i in range(0,len(N_HIDDEN)):
  model = built_model(INPUT_SHAPE,N_HIDDEN[i], NB_CLASSES)
  history = model.fit(X_train, Y_train,
		  batch_size=128, epochs=50,
		  verbose=0, validation_split=0.2)
  # Evaluate the model
  test_loss, test_acc = model.evaluate(X_test, Y_test)
  print(test_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.2995 - accuracy: 0.9195
0.9194999933242798
313/313 [==============================] - 1s 2ms/step - loss: 0.1899 - accuracy: 0.9430
0.9430000185966492
313/313 [==============================] - 1s 2ms/step - loss: 0.1650 - accuracy: 0.9523
0.9523000121116638
313/313 [==============================] - 1s 2ms/step - loss: 0.1452 - accuracy: 0.9563
0.9563000202178955
313/313 [==============================] - 1s 2ms/step - loss: 0.1478 - accuracy: 0.9548
0.954800009727478


<a id='ex_4'></a>
**Exercise 4:** Let us now repeat the same after changing the batch size (minimum 5 different values). Compare the model performance in terms of accuracy. What do you understand from this?

**Answer** 

10: 91.94%
20: 94.30%
30: 95.23%
40: 95.63%
50: 91.94%

The results show that as we increase the number of neurons, we will obtain better results. However, it does not mean that we have increased the number of neuons in the hidden layers abruptly as it could cause an overfitting.


In [33]:
# Task to do choose different values for batch size (minimum five different values)
BATCH_SIZE = [8,16,32,64,128] # Choose a value

In [ ]:
## Do not change anything below
model = built_model(INPUT_SHAPE,128, NB_CLASSES)
history = model.fit(X_train, Y_train,
		batch_size=BATCH_SIZE, epochs=50,
		verbose=1, validation_split=0.2)
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy: {:.2f} %'.format(test_acc*100))

In [34]:
for i in range(0,len(BATCH_SIZE)):
  model = built_model(INPUT_SHAPE,128, NB_CLASSES)
  history = model.fit(X_train, Y_train,
		  batch_size=BATCH_SIZE[i], epochs=50,
		  verbose=0, validation_split=0.2)
  # Evaluate the model
  test_loss, test_acc = model.evaluate(X_test, Y_test)
  print(test_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.0910 - accuracy: 0.9791
0.9790999889373779
313/313 [==============================] - 1s 2ms/step - loss: 0.0799 - accuracy: 0.9788
0.9787999987602234
313/313 [==============================] - 1s 2ms/step - loss: 0.0839 - accuracy: 0.9738
0.973800003528595
313/313 [==============================] - 1s 2ms/step - loss: 0.0890 - accuracy: 0.9728
0.9728000164031982
313/313 [==============================] - 1s 2ms/step - loss: 0.1195 - accuracy: 0.9662
0.9661999940872192


There were no big differences using different values ​​in the BATCH_SIZE parameter. As this parameter increased, the accuracy decreased slightly.

<a id='ex_5'></a>
**Exercise 5:** And now we do the same with different [optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) available in TensorFlow. Change the optimizers and compare the model performance in terms of accuracy. What do you understand from this?

**Answer** Please type your answer here (Double click to edit)

In [43]:
# Task to do choose different optimizers
opt = ['sgd','adam']# Choose from available optimizers

In [42]:
## Do not change anything below
for i in range(0,len(opt)):
  model = built_model(INPUT_SHAPE,128, NB_CLASSES, opt[i])
  history = model.fit(X_train, Y_train,
		  batch_size=128, epochs=50,
		  verbose=0, validation_split=0.2)
  # Evaluate the model
  test_loss, test_acc = model.evaluate(X_test, Y_test)
  print(test_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.1239 - accuracy: 0.9635
0.9635000228881836
313/313 [==============================] - 1s 2ms/step - loss: 0.1264 - accuracy: 0.9808
0.9807999730110168


<a id='ex_O'></a>
### Optional Exercise: Fashion MNIST

Repeat the above exercises (3-5) with different dataset. You can use Fashion MNIST another popular ML dataset. Are the results same? Comment.

To download fashion mnist you can use the following code:

```
fashion_mnist = keras.datasets.fashion_mnist

(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
```

In [44]:
fashion_mnist = keras.datasets.fashion_mnist
 
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [45]:
# Processing the data
assert(len(X_train.shape)==3), "The input data is not of the right shape"
RESHAPED = X_train.shape[1]*X_train.shape[2]

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [46]:
# Data Normalization
X_train, X_test = X_train / 255.0, X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [47]:
INPUT_SHAPE = RESHAPED
NB_CLASSES = len(set(Y_train))

In [48]:
# one-hot encode
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

In [49]:
N_HIDDEN = [10,20,30,40,50]

In [51]:
## Do not change anything below
for i in range(0,len(N_HIDDEN)):
  model = built_model(INPUT_SHAPE,N_HIDDEN[i], NB_CLASSES)
  history = model.fit(X_train, Y_train,
		  batch_size=128, epochs=50,
		  verbose=0, validation_split=0.2)
  # Evaluate the model
  test_loss, test_acc = model.evaluate(X_test, Y_test)
  print(test_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.4488 - accuracy: 0.8429
0.8428999781608582
313/313 [==============================] - 1s 2ms/step - loss: 0.4230 - accuracy: 0.8515
0.8514999747276306
313/313 [==============================] - 1s 2ms/step - loss: 0.4228 - accuracy: 0.8521
0.8521000146865845
313/313 [==============================] - 1s 2ms/step - loss: 0.4029 - accuracy: 0.8564
0.8564000129699707
313/313 [==============================] - 1s 2ms/step - loss: 0.4003 - accuracy: 0.8610
0.8610000014305115


When comparing different numbers of neurons for this new data set, it is seen that the accuracy decreased compared to the results obtained with the previous data set.